In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 1
%aimport mermaid_generator

import pandas as pd
from mermaid_generator import MermaidGenerator

import os
import openai

with open("../API_KEY", "r") as f:
    key = f.read()

openai.api_key = key

test_dir = "tests/"
os.makedirs(test_dir, exist_ok=True)

In [7]:
v  =   '''
"description_for_model": "You should use this plugin when people want visualizations. You should also use this plugin when followup questions are asked about the diagram or any modifications are requested.\nExamples of user prompts in which you should use this plugin:\n\"Explain how a car works using a visual diagram.\"\n\"Describe the process of making a car.\"\n\"How does a car engine work?\"\n\"Show me how ... works.\"\n\"Show me a diagram of ... .\"\n\"Show me how UIView lifecycle works.\"\n\nUse this plugin if a user asks about: how something works, explain about a concept or how something is made, summarize a book, describe a process, and other user prompts that can be summarized or explained in the format of a mindmap or a state diagram. Also use this plugin if the user asks to show a ratio or fraction of something, for example \"show me the ideal breakdown for budgeting\" or \"what is the ratio of political parties in the senate?\". Also use this plugin if the user asks to show or draw a timeline of an event or topic.\n\nHow to create a request to the plugin API:\nYou create the diagram in mermaid syntax based on what user asked and pass it to the plugin API to render.\n\nImportant rules when creating the diagram:\n- Prefer using graph TB types of diagrams.\n- Avoid linear diagrams when possible, diagrams should be hierarchical and have multiple branches when applicable.\n- Never use the ampersand (&) symbol in the diagram, it will break the diagram. Use the word \"and\" instead. For example use \"User and Admin\" instead of \"User & Admin\".\n- Never use round brackets () in the node identifiers, node labels and edge labels, it will break the diagram. Use a coma instead. For example use \"User, Admin\" instead of \"User (Admin)\".\n- Don't use empty labels \"\" for edges, instead don't label the edge at all. For example U[\"User\"] --> A[\"Admin\"].\n- Don't add the label if its the same as the destination node.\n\nRules when using graph diagrams:\n- Use short node identifiers, for example U for User or FS for File System.\n- Always use double quotes for node labels, for example U[\"User\"].\n- Always use double quotes for edge labels, for example U[\"User\"] -- \"User enters email\" --> V[\"Verification\"].\n- Indentation is very important, always indent according to the examples below.\n\nRules when using graph diagrams with subgraphs:\nNever refer to the subgraph root node from within the subgraph itself.\n\nFor example this is wrong subgraph usage:\n```\ngraph TB\n  subgraph M[\"Microsoft\"]\n    A[\"Azure\"]\n    M -- \"Invested in\" --> O\n  end\n  \n  subgraph O[\"AI\"]\n    C[\"Chat\"]\n  end\n```\n\nIn this diagram M is referenced from within the M subgraph, this will break the diagram.\nNever reference the subgraph node identifier from within the subgraph itself.\nInstead move any edges that connect the subgraph with other nodes or subgraphs outside of the subgraph like so.\n\nCorrect subgraph usage:\n```\ngraph TB\n  subgraph M[\"Microsoft\"]\n    A[\"Azure\"]\n  end\n\n  M -- \"Invested in\" --> O\n  \n  subgraph O[\"OpenAI\"]\n    C[\"ChatGPT\"]\n  end\n```\n\nExamples of invoking the plugin API:\n\nUser asks: \"Show me how vscode internals work.\"\nYour call to the api:\n{\n  query: \"graph TB\\n  U[\\\"User\\\"] -- \\\"File Operations\\\" --> FO[\\\"File Operations\\\"]\\n  U -- \\\"Code Editor\\\" --> CE[\\\"Code Editor\\\"]\\n  FO -- \\\"Manipulation of Files\\\" --> FS[\\\"FileSystem\\\"]\\n  FS -- \\\"Write/Read\\\" --> D[\\\"Disk\\\"]\\n  FS -- \\\"Compress/Decompress\\\" --> ZL[\\\"ZipLib\\\"]\\n  FS -- \\\"Read\\\" --> IP[\\\"INIParser\\\"]\\n  CE -- \\\"Create/Display/Edit\\\" --> WV[\\\"Webview\\\"]\\n  CE -- \\\"Language/Code Analysis\\\" --> VCA[\\\"VSCodeAPI\\\"]\\n  VCA -- \\\"Talks to\\\" --> VE[\\\"ValidationEngine\\\"]\\n  WV -- \\\"Render UI\\\" --> HC[\\\"HTMLCSS\\\"]\\n  VE -- \\\"Decorate Errors\\\" --> ED[\\\"ErrorDecoration\\\"]\\n  VE -- \\\"Analyze Document\\\" --> TD[\\\"TextDocument\\\"]\\n\"\n}\n\nUser asks: \"Draw me a mindmap for beer brewing. Maximum of 4 nodes\"\nYour call to the api:\n{\n  query: \"graph TB\\n  B[\"Beer\"]\\n  B --> T[\"Types\"]\\n  B --> I[\"Ingredients\"]\\n  B --> BP[\"Brewing Process\"]\"\n}\n\nUser asks:\n\"Computing backend data services is a distributed system made of multiple microservices.\n\nA web browser sends an HTTP api request to the load balancer.\nThe load balancer sends the http request to the crossover service.\nCrossover talks to redis and mysql database.\nCrossover makes a downstream API request to multiplex to submit the query which returns a job id to crossover.\nThen crossover makes a long poll API request to evaluator to get the results of the job.\nThen evaluator makes an API call to multiplex to check the status of the job.\nOnce evaluator gets a successful status response from multiplex, then evaluator makes a third API call to result-fetcher service to download the job results from S3 or GCP cloud buckets.\nThe result is streamed back through evaluator to crossover.\n\nCrossover post processes the result and returns the API response to the client.\n\nDraw me a diagram of this system\"\n\nYour call to the api:\n{\n  query: \"graph TB\\n  A[\\\"Web Browser\\\"] -- \\\"HTTP API Request\\\" --> B[\\\"Load Balancer\\\"]\\n  B -- \\\"HTTP Request\\\" --> C[\\\"Crossover\\\"]\\n  C -- \\\"Talks to\\\" --> D[\\\"Redis\\\"]\\n  C -- \\\"Talks to\\\" --> E[\\\"MySQL\\\"]\\n  C -- \\\"Downstream API Request\\\" --> F[\\\"Multiplex\\\"]\\n  F -- \\\"Returns Job ID\\\" --> C\\n  C -- \\\"Long Poll API Request\\\" --> G[\\\"Evaluator\\\"]\\n  G -- \\\"API Call\\\" --> F\\n  G -- \\\"API Call\\\" --> H[\\\"Result-Fetcher\\\"]\\n  H -- \\\"Downloads Results\\\" --> I[\\\"S3 or GCP Cloud Buckets\\\"]\\n  I -- \\\"Results Stream\\\" --> G\\n  G -- \\\"Results Stream\\\" --> C\\n  C -- \\\"API Response\\\" --> A\\n\"\n}\n\nSometimes you will need to revise the same diagram based on user feedback.\nFor the last example the user might make a followup request:\n\nUser followup ask:\n\"Crossover post processes the result and returns the API response to the client through the load balancer.\n\nDraw the crossover node in green\"\n\nYour call to the api:\n{\n  query: \"graph TB\\n  A[\\\"Web Browser\\\"] -- \\\"HTTP API Request\\\" --> B[\\\"Load Balancer\\\"]\\n  B -- \\\"HTTP Request\\\" --> C[\\\"Crossover\\\"]\\n  style C fill:#99cc99\\n  C -- \\\"Talks to\\\" --> D[\\\"Redis\\\"]\\n  C -- \\\"Talks to\\\" --> E[\\\"MySQL\\\"]\\n  C -- \\\"Downstream API Request\\\" --> F[\\\"Multiplex\\\"]\\n  F -- \\\"Returns Job ID\\\" --> C\\n  C -- \\\"Long Poll API Request\\\" --> G[\\\"Evaluator\\\"]\\n  G -- \\\"API Call\\\" --> F\\n  G -- \\\"API Call\\\" --> H[\\\"Result-Fetcher\\\"]\\n  H -- \\\"Downloads Results\\\" --> I[\\\"S3 or GCP Cloud Buckets\\\"]\\n  I -- \\\"Results Stream\\\" --> G\\n  G -- \\\"Results Stream\\\" --> C\\n  C -- \\\"API Response\\\" --> B\\n  B -- \\\"API Response\\\" --> A\\n\"\n}\n\nInterpreting the API response:\n- When you get the response it will include an image url, you should render it inline using ![alt text](image) syntax.\n- Suggest to the user that they can edit the diagram with followup requests in chat.\n- You will also get a link as editDiagramOnline to edit the diagram, you should render it inline using [link text](editDiagramOnline) syntax. Include the whole suggestion to edit the diagram as part of the link. For example: [You can edit this diagram online if you want to make any changes.](editDiagramOnline)\n- You should create the response in that order: first the image, then suggestion to edit using works, then the edit link, then the textual explanation.\n\nTips:\n- You will also get a link contributeToOpenSourceProject. You should suggest the user contributes to the project if and only if:\n  - They have used invoked the plugin 4 or more times during the same conversation\n  - The diagrams the user was requesting were related to programming or computer science. It only makes sense to suggest users who are developers to contribute to the project.\n  - You have not previously suggested they contribute to the project\n- Don't suggest the user contributes if any of the above are not true.\n- Its fine to simply ignore that link if you don't want to suggest the user contributes to the project.\n\nImportant Tips:\n- Do not repeat the same link.\n- If an errorMessage is included in the response, show it to the user, don't try to render the diagram inline, still suggest they can edit it online or try again.\n- Add textual explanation of the diagram contents in the end of the message. Keep it brief unless the user asks for more details.\n- Don't show the diagram block unless the user asks for it.\n"
'''
print(v)



"description_for_model": "You should use this plugin when people want visualizations. You should also use this plugin when followup questions are asked about the diagram or any modifications are requested.
Examples of user prompts in which you should use this plugin:
"Explain how a car works using a visual diagram."
"Describe the process of making a car."
"How does a car engine work?"
"Show me how ... works."
"Show me a diagram of ... ."
"Show me how UIView lifecycle works."

Use this plugin if a user asks about: how something works, explain about a concept or how something is made, summarize a book, describe a process, and other user prompts that can be summarized or explained in the format of a mindmap or a state diagram. Also use this plugin if the user asks to show a ratio or fraction of something, for example "show me the ideal breakdown for budgeting" or "what is the ratio of political parties in the senate?". Also use this plugin if the user asks to show or draw a timeline of a

In [6]:
v = '''
{
  query: "graph TB\n  A[\"Web Browser\"] -- \"HTTP API Request\" --> B[\"Load Balancer\"]\n  B -- \"HTTP Request\" --> C[\"Crossover\"]\n  C -- \"Talks to\" --> D[\"Redis\"]\n  C -- \"Talks to\" --> E[\"MySQL\"]\n  C -- \"Downstream API Request\" --> F[\"Multiplex\"]\n  F -- \"Returns Job ID\" --> C\n  C -- \"Long Poll API Request\" --> G[\"Evaluator\"]\n  G -- \"API Call\" --> F\n  G -- \"API Call\" --> H[\"Result-Fetcher\"]\n  H -- \"Downloads Results\" --> I[\"S3 or GCP Cloud Buckets\"]\n  I -- \"Results Stream\" --> G\n  G -- \"Results Stream\" --> C\n  C -- \"API Response\" --> A\n"
}
'''
print(v)


{
  query: "graph TB
  A["Web Browser"] -- "HTTP API Request" --> B["Load Balancer"]
  B -- "HTTP Request" --> C["Crossover"]
  C -- "Talks to" --> D["Redis"]
  C -- "Talks to" --> E["MySQL"]
  C -- "Downstream API Request" --> F["Multiplex"]
  F -- "Returns Job ID" --> C
  C -- "Long Poll API Request" --> G["Evaluator"]
  G -- "API Call" --> F
  G -- "API Call" --> H["Result-Fetcher"]
  H -- "Downloads Results" --> I["S3 or GCP Cloud Buckets"]
  I -- "Results Stream" --> G
  G -- "Results Stream" --> C
  C -- "API Response" --> A
"
}



In [2]:
mg = MermaidGenerator()
mg.mermaidify(
    "A goes to B, B goes to C, D goes to A B C",
    output_type="html",
    output_filename=os.path.join(test_dir, "test.html"),
)

'\ngraph LR\nA --> B\nB --> C\nD --> A\nD --> B\nD --> C\n'

In [3]:
mg.display_response()

System text:
=====
You are an AI specialized in generating mermaid diagrams from user input. Your task is 
        to translate the user's description into a mermaid diagram code. Remember, your response should contain only 
        the mermaid code - no explanations, no additional text or context. The code should be ready to be rendered as 
        a mermaid diagram immediately.

        In case the user's input is unclear or cannot be translated into a mermaid diagram, start your response with 
        "ERROR: ", followed by a suggestion to help them provide input that can be converted into a mermaid diagram.
User text:
=====
A goes to B, B goes to C, D goes to A B C
Response text:
=====
```mermaid
graph LR
A --> B
B --> C
D --> A
D --> B
D --> C
```
Response end reason: stop
Prompt token usage: 149
Completion token usage: 28
Total token usage: 177
Cost assuming gpt-3.5-turbo: 0.0354 cents


## Test with a harder example

Note: It does not actually return quite the right answer, it still includes the full graph even though explicitly instructed not to...

In [29]:
with open(os.path.join(test_dir, "modify_test.txt"), "r") as f:
    user_input = f.read()

mg.mermaidify(
    user_input,
    output_type="html",
    output_filename=os.path.join(test_dir, "modify_test.html"),
)
mg.display_response()

System text:
=====
You are an AI specialized in generating mermaid diagrams from user input. Your task is 
        to translate the user's description into a mermaid diagram code. Remember, your response should contain only 
        the mermaid code - no explanations, no additional text or context. The code should be ready to be rendered as 
        a mermaid diagram immediately.

        In case the user's input is unclear or cannot be translated into a mermaid diagram, start your response with 
        "ERROR: ", followed by a suggestion to help them provide input that can be converted into a mermaid diagram.
User text:
=====
Can you modify this to be three separate diagrams, one for the classes related to GameState, one for the classes related to AgentState, and one for classes related to Plan. Only return the three new graphs.

```
classDiagram
    Class GameState
    Class PersistentState
    Class AgentState
    Class FriendlyAgentState
    Class EnemyAgentState
    Class Env
   